# Overview
This notebook explores the cleaned ramen-ratings CSV. Any engineered features or scripts will be added to explore.py.

# Findings
1. Ramen packaging and five-star ratings are independent.
2. Ramen country of origin and five-star ratings **have a dependent relationship.**

In [1]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle

In [2]:
# create train split for exploration
train, _, _ = wrangle.prep_explore()
print('')

# check work
train.info()

Train size: (1515, 5) Validate size: (506, 5) Test size: (506, 5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 2566 to 2127
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand       1515 non-null   object
 1   name        1515 non-null   object
 2   package     1515 non-null   object
 3   country     1515 non-null   object
 4   five_stars  1515 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 60.7+ KB


# Initial Exploration of Ramen Packaging
Let's check if there's a dependent relationship between packaging and our target.

Hypotheses:
- $H_0$: Packaging and five-star ratings are independent.
- $H_a$: Packaging and five-star ratings have a dependent relationship.

Confidence interval: 95%

In [3]:
# set confidence interval
alpha = .05

In [4]:
# check dependence of packaging and target
package_5star_crosstab = pd.crosstab(train.package, train.five_stars)
_, p, _, _ = stats.chi2_contingency(package_5star_crosstab)

In [5]:
# check if p is significant
if p < alpha:
    print("Packaging and five-star ratings have a dependent relationship with 95% confidence.")
    print("p-value:", p)
else:
    print("Packaging and five-star ratings are independent, did not pass 95% confidence interval.")
    print("p-value:", p)

Packaging and five-star ratings are independent, did not pass 95% confidence interval.
p-value: 0.43924606238117814


**Packaging and five-star ratings are independent.** We will not use 'package' in our predictive model.

# Initial Exploration of Ramen Country of Origin
Let's check if there's a dependent relationship between country and our target.

Hypotheses:
- $H_0$: Country of origin and five-star ratings are independent.
- $H_a$: Country of origin and five-star ratings have a dependent relationship.

Confidence interval: 95%

In [6]:
# set confidence interval
alpha = .05

In [7]:
# create crosstab for chi-square statistical test
country_5star_crosstab = pd.crosstab(train.country, train.five_stars)
# limit only to countries with sufficient value counts in crosstab (an assumption of chi-square)
enough_values_mask = (country_5star_crosstab[False] > 5) & (country_5star_crosstab[True] > 5)
# run chi-square test
_, p, _, _ = stats.chi2_contingency(country_5star_crosstab[enough_values_mask])

In [8]:
# check if p is significant
if p < alpha:
    print("Country of origin and five-star ratings have a dependent relationship with 95% confidence.")
    print("p-value:", p)
else:
    print("Country of origin and five-star ratings are independent, did not pass 95% confidence interval.")
    print("p-value:", p)

Country of origin and five-star ratings have a dependent relationship with 95% confidence.
p-value: 1.680501270556502e-06


**Country of origin and five-star ratings have a dependent relationship.** We will further explore country of origin and consider using it in our model.

# Initial Exploration of Ramen Brand
Let's check if there's a dependent relationship between ramen brand and our target.

Hypotheses:
- $H_0$: Ramen brand and five-star ratings are independent.
- $H_a$: Ramen brand and five-star ratings have a dependent relationship.

Confidence interval: 95%

In [9]:
# set confidence interval
alpha = .05

In [10]:
# create crosstab for chi-square statistical test
brand_5star_crosstab = pd.crosstab(train.brand, train.five_stars)
# limit only to brands with sufficient value counts in crosstab (an assumption of chi-square)
enough_values_mask = (brand_5star_crosstab[False] > 5) & (brand_5star_crosstab[True] > 5)
# run chi-square test
_, p, _, _ = stats.chi2_contingency(brand_5star_crosstab[enough_values_mask])

In [11]:
# check if p is significant
if p < alpha:
    print("Ramen brand and five-star ratings have a dependent relationship with 95% confidence.")
    print("p-value:", p)
else:
    print("Ramen brand and five-star ratings are independent, did not pass 95% confidence interval.")
    print("p-value:", p)

Ramen brand and five-star ratings are independent, did not pass 95% confidence interval.
p-value: 0.22292206264805428


**Ramen brand and five-star ratings are independent.** We will not use 'brand' in our predictive model.

# Breaking Down the Ramen Product Name
Nearly all ramen reviews in our dataset have a unique combination of brand and product name. Only 21 combinations of brand and product out of the nearly 1500 in our exploration split have two reviews, and there are no combinations with more than two reviews. Because of this low commonality, we can't run initial chi-square tests to see if product names have a dependent relationship with five star reviews.

In [12]:
# show the review repeats and non-repeats
print("Brand+name with only one review:", (train[['brand','name']].value_counts() == 1).sum())
print("Brand+name with two reviews:", (train[['brand','name']].value_counts() == 2).sum())
print("Brand+name with more than two reviews:", (train[['brand','name']].value_counts() > 2).sum())

Brand+name with only one review: 1473
Brand+name with two reviews: 21
Brand+name with more than two reviews: 0


Before we can understand the relationship between product name and the target, we will need to split out certain keywords in product names to use as features.

## Identifying Keywords to Use
### Results
I looked at ramen with the word 'flavor' in the title in my older analysis. These are the counts I found:
- Chicken (48); Beef (30); Pork (16); Shrimp (12); Fish/Seafood (10); Chow Mein (6);
- Chili/Chilli (7); Curry (6); Kimchi (5); Stir-Fry (4);
- Tom Yum (6); Tonkotsu (5); Miso (4); Wonton (2); Jjamppong (1); Shiodare (1); Bulgogi (1); Bulalo (1);
- Tomato (6); Mushroom (5); Sesame (5); Onion (3); Shiitake (1);
- Spicy (26); Hot (12); Soy (6); Sour (6); Sweet (2); Umami [savory] (2);
- Sriracha (3); 
- Abalone (4);
- Vegetable (11);
- has_Lime (8);

I will use these words to search the train split for more potential keywords. **The words I find interesting will all be listed here:**
- **Instant or Minute** (does the name indicate a worse rating?) - **258 reviews**
    * "Instant" used by many brands
    * "Minute" used by three brands (Maggi, Tasy Bite, Trident - merge under 'Instant')
- **Artificial, Imitation, or Simulated** (does the name indicate a worse rating?) - **93 reviews**
    * Artificial (Many brands use this word)
    * Imitation (Gefen and Traditional use this word - merge under 'Artificial')
    * Simulated (Mr. Noodles brand uses this word - merge under 'Artificial')
- **Good or Premium or Delicious** (does the name indicate a better rating?) - **Not enough reviews** (19 reviews)
- **Less Sodium** (does less sodium make for a worse-rated product?) - **Not enough reviews** (5 reviews)

### Scratch Work to Find Keywords (cells ran multiple times with different inputs)

In [13]:
# chicken (highest word count in previous analysis)
# train[train.name.str.contains('Chicken')].name.value_counts().head(10)

In [14]:
# beef (second highest word count in previous analysis)
# train[train.name.str.contains('Beef')].name.value_counts().head(10)

In [15]:
# spicy (third highest word count in previous analysis)
# train[train.name.str.contains('Spicy')].name.value_counts().head(10)

In [16]:
# artificial, imitation, or simulated
mask = (train.name.str.contains('Artificial') |
        train.name.str.contains('Imitation') |
        train.name.str.contains('Simulated'))
# train[mask].name.value_counts().sample(10, random_state=777)

In [17]:
# instant or minute
mask = (train.name.str.contains('Instant') |
        train.name.str.contains('Minute'))
# train[mask].name.value_counts().head(10)

In [18]:
# good or premium or delicious
mask = (train.name.str.contains('Good') |
        train.name.str.contains('Premium') |
        train.name.str.contains('Delicious'))
# train[mask].name.value_counts().head(10)

In [19]:
# sodium
# train[train.name.str.contains('Sodium')]

## Creating Groups of Similar Keywords
From the product names, there are a few stand-out themes:
- Product name describes flavor
- Product name indicates flavoring is artificial
- Product name describes fast cook time

"Artificial" and "Instant" *only require one feature each to describe the product.* **The flavor, though, will require multiple features** to describe the ramen due to the fact that a ramen can have one or multiple flavors.

> Note: Most ramen seems to be 'instant', but doesn't explicitly say so. Because of this, we should not use 'instant' as a feature.

Another issue is that some product names use English and some do not use English. In order to accomodate this, **we will need to translate some words** to English and include them in our features (EX: put "soy" and "shoyu"/"shouyu" into one feature).

#### Translations
- Words indicating Vegetable: 'Vegetable', 'Soon Veggie'
- Words indicating Beef: 'Beef', 'Gomtang', 'Seolleongtang'
- Words indicating Pork: 'Jjajangmen', 'Jiajang', 'Tonkotsu'
- Words indicating Sausage: 'Budae'
- Words indicating Seafood: 'Seafood', 'Crab', 'Anchovy', 'Bajirak', 'Clam'
- Words indicating Noodles: 'Noodle', 'Myeon', 'Myon'
- Words indicating Rice Cake: 'Tteok', 'Rabokki', 'Raobokki'
- Words indicating Soup: 'Soup', 'Jjigae', 'Consomme'
- Words indicating Instant: 'Instant', 'Minute', 'Ramyun', 'Jinjja', 'Bibim'
- Words indicating Spicy: 'Spicy', 'Spice', 'Shin', 'Jjamppong'/'Jjambbong'(seafood), 'Buldalk'(chicken), 'Sutah'(beef), 'Budae'(sausage), 'Habanero', 'Jinjja', 'Jin', 'Yeul', 'Mala', 'Teumsae', 'Bibim', 'Picante', 'Bulnak', 'Volcano', 'Odongtong', 'Sriracha'
- Words indicating Stir-Fry: 'Stir Fry', 'Bokkeum', 'Tteokbokki'/'Topokki', 'Yukgaejang', 'Rabokki'
- Words indicating no-spice: 'Ramyonsari'(plain), 'Keopnurungji', 
- Words indicating Kimchi: 'Sabalmyeon'

### Artificial

In [20]:
# create boolean mask
mask = (train.name.str.contains('Artificial') |
        train.name.str.contains('Imitation') |
        train.name.str.contains('Simulated'))
# add column indicating the presence of these words in the product name
train['artificial'] = mask
# check work
train.head(3)

,brand,name,package,country,five_stars,artificial
2566,Samyang,Hot,Pack,South Korea,False,False
332,Nongshim,Shin Noodle Soup,Cup,USA,True,False
1363,Doll,Hello Kitty Dim Sum Noodle Japanese Curry Flavour,Cup,Hong Kong,False,False


### Instant
> Note: Most ramen seems to be 'instant', but doesn't explicitly say so. Because of this, we should not use 'instant' as a feature.

In [21]:
# # instant, minute, ramyun
# mask = (train.name.str.contains('Instant') |
#         train.name.str.contains('Minute') | 
#         train.name.str.contains('Ramyun'))
# # add column indicating the presence of these words in the product name
# train['instant'] = mask
# # check work
# train.head(3)

### Spicy

In [22]:
# print a list of countries with a ramen product
print(train.country.unique().tolist())

['South Korea', 'USA', 'Hong Kong', 'UK', 'Thailand', 'Japan', 'Taiwan', 'Malaysia', 'India', 'Canada', 'Singapore', 'Philippines', 'China', 'Mexico', 'Indonesia', 'Cambodia', 'Netherlands', 'Australia', 'Nepal', 'Vietnam', 'Myanmar', 'Germany', 'Pakistan', 'Hungary', 'Colombia', 'Bangladesh', 'Brazil']


In [23]:
# cell for exploring each ramen product's names, translating words I see in the list
train[train.country == 'Hong Kong'].name.tolist()

['Hello Kitty Dim Sum Noodle Japanese Curry Flavour',
 'Noodle King Hong Koing Style Wonton',
 'Demae Iccho Mushroom With Vegetables Flavour Macaroni',
 'Demae Iccho Black Garlic Oil Tonkotsu Flavour Instant Noodles',
 'Cup Noodles Spicy Seafood',
 'Demae Ramen Bar Noodle Aka Tonkotsu Flavour Instant Noodle',
 'Big Bowl Crab ',
 'Big Cup Noodles Tonkotsu Flavour',
 'Taisho Fried Noodle Sesame Paste Flavour Instant Noodle',
 'Demae Iccho Red Hot Seafood Flavour Instant Noodle',
 'Spicy Artificial Pork',
 'Demae Iccho XO Sauce Seafood Flavour',
 'E-Men Noodles Abalone Soup Flavor',
 'Instant Noodle Preserved Vegetable Flavour',
 'Demae Iccho Chicken Flavour Instant Noodle (Bowl Noodle)',
 'Big Bowl Artificial Chicken',
 'Spicy Artificial Beef',
 'Pickled Vegetable',
 'Seaweed Hot & Spicy Instant Noodle Mushroom & Beef Flavour',
 'Demae Ramen Roast Beef Flavour Instant Noodles',
 'Noodle King Lobster Thick',
 'Nupasta Bacon In Carbonara Sauce Flavour Instant Noodle',
 'Demae Iccho Spicy T

In [24]:
# spicy, shin, jjamppong, buldalk
train[train.name.str.contains('Shin')]

,brand,name,package,country,five_stars,artificial
332,Nongshim,Shin Noodle Soup,Cup,USA,True,False
2157,Noodle Time,Shin Cup,Cup,South Korea,False,False
291,Myojo,Shin Toyama Black Ramen,Bowl,Japan,False,False
1903,Nongshim,Shin Ramyun Shrimp,Pack,China,False,False
2002,Nongshim,Shin Ramyun Black,Pack,South Korea,False,False
1017,Nongshim,Shin Ramyun Noodle Soup (New Edition),Pack,USA,False,False
2289,Nongshim,Shin Bowl,Bowl,South Korea,False,False
47,Nongshim,Shin Ramyun Black,Pack,South Korea,True,False
1582,Nongshim,Shin Ramyun Black Spicy Beef,Cup,South Korea,False,False
1402,Nongshim,Shin Black Spicy Pot-au-feu Flavor Premium Noo...,Cup,USA,True,False
